In [36]:
import requests  # Para peticiones HTTP
import json      # Para trabajar con JSON
import pandas as pd  # Para DataFrames
from numpy import nan
from IPython.display import display  # Para mostrar resultados en Jupyter

# Tu API key de RapidAPI para AeroDataBox
flights_key = "6398d16efdmshfb022c9141871dcp15f1b7jsn115de15bb173"


In [37]:
# URL de la API para vuelos de llegada a Casablanca (ICAO: GMMN)
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/GMMN/2025-09-01T08:00/2025-09-01T20:00"

# Parámetros de la API
querystring = {
    "withLeg": "true",
    "direction": "Arrival",
    "withCancelled": "true",
    "withCodeshared": "true",
    "withCargo": "true",
    "withPrivate": "true",
    "withLocation": "false"
}

# Headers con tu API key
headers = {
    "x-rapidapi-host": "aerodatabox.p.rapidapi.com",
    "x-rapidapi-key": flights_key
}


In [38]:
response = requests.get(url, headers=headers, params=querystring)

if response.status_code == 200:
    flights_data = response.json()
    
    # Guardar en data.json
    try:
        with open("data.json", "r") as f:
            existing_data = json.load(f)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []

    existing_data.append(flights_data)

    with open("data.json", "w") as f:
        json.dump(existing_data, f, indent=4)
        print("✅ Data appended to data.json file.")
else:
    print("❌ Failed to retrieve data from the API. Status code:", response.status_code)


✅ Data appended to data.json file.


In [39]:
# Guardar los vuelos de llegada
flight_arrivals = flights_data.get("arrivals", [])

# Crear lista con los campos importantes
flights_list = []
for flight in flight_arrivals:
    departure_airport = flight.get("departure", {}).get("airport", {})
    departure_country = departure_airport.get("country", {}).get("iataCode", nan)
    
    arrival_info = flight.get("arrival", {}).get("scheduledTime", {})
    
    flight_info = {
        "flight_number": flight.get("number", nan),
        "from": departure_airport.get("name", nan),
        "from_country": departure_country,
        "to": "Casablanca",  # Destino fijo
        "to_country": "MA",  # País fijo
        "airline": flight.get("airline", {}).get("name", nan),
        "aircraft": flight.get("aircraft", {}).get("model", nan),
        "Scheduled_arrival_time.utc": arrival_info.get("utc", nan),
        "Scheduled_arrival_time.local": arrival_info.get("local", nan)
    }
    flights_list.append(flight_info)



In [40]:
# Crear DataFrame
df_flights = pd.DataFrame(flights_list)

# Mostrar las primeras filas
display(df_flights.head())



,flight_number,from,from_country,to,to_country,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,AT 512,Lomé,NaN,Casablanca,MA,Royal Air Maroc,Boeing 737-800,2025-09-01 07:20Z,2025-09-01 08:20+01:00
1,AT 1401,Oujda,NaN,Casablanca,MA,Royal Air Maroc,Boeing 737-800 (winglets),2025-09-01 07:25Z,2025-09-01 08:25+01:00
2,AT 1447,Errachidia,NaN,Casablanca,MA,Royal Air Maroc,Embraer 190,2025-09-01 07:30Z,2025-09-01 08:30+01:00
3,AT 558,Cotonou,NaN,Casablanca,MA,Royal Air Maroc,Boeing 737-800,2025-09-01 07:40Z,2025-09-01 08:40+01:00
4,AT 219,Washington,NaN,Casablanca,MA,Royal Air Maroc,Boeing 787-8,2025-09-01 08:20Z,2025-09-01 09:20+01:00


In [4]:
# 1. RECOLECCIÓN Y PROCESAMIENTO DE DATOS
import requests
import pandas as pd
import numpy as np
import json
import sqlalchemy
import pymysql

# --- CLAVE API (Tu clave de OpenWeatherMap) ---
OWM_key = "d0f554be7cf5fcb3cfef63b8267fff55" 

# --- DATOS DE CASABLANCA ---
city = "Casablanca"
country = "MA" 

# Solicitud a la API de OpenWeatherMap
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')
data = response.json()

# Verificar que la solicitud fue exitosa
if response.status_code != 200:
    print(f"❌ Error al obtener datos de la API. Código de estado: {response.status_code}")
else:
    # Extract forecast list
    forecast_list = data.get('list', [])

    # Preparar listas para DataFrame
    times = []
    temperatures = []
    humidities = []
    weather_statuses = []
    wind_speeds = []
    rain_volumes = []
    snow_volumes = []

    for entry in forecast_list:
        times.append(entry.get('dt_txt', np.nan))
        temperatures.append(entry.get('main', {}).get('temp', np.nan))
        humidities.append(entry.get('main', {}).get('humidity', np.nan))
        weather_statuses.append(entry.get('weather', [{}])[0].get('main', np.nan))
        wind_speeds.append(entry.get('wind', {}).get('speed', np.nan))
        rain_volumes.append(entry.get('rain', {}).get('3h', np.nan))
        snow_volumes.append(entry.get('snow', {}).get('3h', np.nan))

    # Crear DataFrame
    df = pd.DataFrame({
        'weather_datetime': times,
        'temperature': temperatures,
        'humidity': humidities,
        'weather_status': weather_statuses,
        'wind': wind_speeds,
        'rain_qty': rain_volumes,
        'snow': snow_volumes,
        'municipality_iso_country': f"{city},{country}"})

    print("✅ Datos de Casablanca cargados en DataFrame:")
    print(df.head())

    # 2. CONEXIÓN Y TRANSFERENCIA A MYSQL (CORREGIDO)

    # ---  CREDENCIALES DE MYSQL ---
    schema = "Actividad9"  # <--- ¡CORRECCIÓN A LA 'A' MAYÚSCULA!
    host = "127.0.0.1"
    user = "root"
    password = "PepeCimf#666" 
    port = 3306
    TABLE_NAME = 'weather_data' 

    # Crear la cadena de conexión con el conector PyMySQL
    con_url = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    try:
        # Crear el motor de conexión
        engine = sqlalchemy.create_engine(con_url)
        
        # Enviar los datos del clima a la base de datos
        df.to_sql(TABLE_NAME, 
                  if_exists='append', # Añadir a la tabla existente
                  con=engine, 
                  index=False)
        
        print("\n✅ ¡Datos del clima transferidos exitosamente a MySQL!")
        
    except Exception as e:
        print(f"\n❌ ¡ERROR en la transferencia a MySQL! Asegúrate que tu servidor esté encendido.")
        print(f"Detalle del error: {e}")

✅ Datos de Casablanca cargados en DataFrame:
      weather_datetime  temperature  humidity weather_status  wind  rain_qty  \
0  2025-10-30 21:00:00        20.24        86         Clouds  0.82       NaN   
1  2025-10-31 00:00:00        19.83        84         Clouds  1.95       NaN   
2  2025-10-31 03:00:00        19.47        80         Clouds  1.45       NaN   
3  2025-10-31 06:00:00        19.17        84         Clouds  1.37       NaN   
4  2025-10-31 09:00:00        20.68        76         Clouds  1.10       NaN   

   snow municipality_iso_country  
0   NaN            Casablanca,MA  
1   NaN            Casablanca,MA  
2   NaN            Casablanca,MA  
3   NaN            Casablanca,MA  
4   NaN            Casablanca,MA  

✅ ¡Datos del clima transferidos exitosamente a MySQL!


In [5]:
# 3. DATOS SIMULADOS PARA EVITAR EL ERROR 403 (PARTE 4)
import pandas as pd
import sqlalchemy
import pymysql
from numpy import nan

# Datos de Vuelos de Ejemplo para Casablanca (GMMN)
# Estos datos tienen las columnas necesarias para la inserción en flight_arrival
flights_data = [
    {
        'scheduled_arrival_time': '2025-09-01T10:30:00',
        'flight_number': 'AT675',
        'from_airport': 'Paris - Orly Airport',
        'airline': 'Royal Air Maroc',
        'aircraft': 'Boeing 737-800',
        'icao_code': 'GMMN'
    },
    {
        'scheduled_arrival_time': '2025-09-01T12:45:00',
        'flight_number': 'LH1426',
        'from_airport': 'Frankfurt Airport',
        'airline': 'Lufthansa',
        'aircraft': 'Airbus A320neo',
        'icao_code': 'GMMN'
    },
    {
        'scheduled_arrival_time': '2025-09-01T15:10:00',
        'flight_number': 'AF1024',
        'from_airport': 'Paris - Charles de Gaulle Airport',
        'airline': 'Air France',
        'aircraft': 'Airbus A330-200',
        'icao_code': 'GMMN'
    }
]

# Crear el DataFrame directamente desde los datos simulados
df_flights = pd.DataFrame(flights_data)

print("✅ Datos de vuelos simulados cargados en DataFrame:")
print(df_flights)


# 4. CONEXIÓN Y CARGA A MYSQL

# ---  CREDENCIALES DE MYSQL ---
# Usamos la base de datos 'gans' que creaste.
schema = "gans"  
host = "127.0.0.1"
user = "root"
password = "PepeCimf#666" 
port = 3306
TABLE_NAME = 'flight_arrival'

# Crear la cadena de conexión con el conector PyMySQL
con_url = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

try:
    # Crear el motor de conexión
    engine = sqlalchemy.create_engine(con_url)
    
    # Enviar los datos de vuelos a la base de datos
    df_flights.to_sql(TABLE_NAME, 
              if_exists='append', 
              con=engine, 
              index=False)
    
    print(f"\n✅ ¡Datos de {len(df_flights)} vuelos transferidos exitosamente a MySQL en la tabla '{TABLE_NAME}'!")
    
except Exception as e:
    print(f"\n❌ ¡ERROR en la transferencia a MySQL! Asegúrate que tu servidor esté encendido.")
    print(f"Detalle del error: {e}")

✅ Datos de vuelos simulados cargados en DataFrame:
  scheduled_arrival_time flight_number                       from_airport  \
0    2025-09-01T10:30:00         AT675               Paris - Orly Airport   
1    2025-09-01T12:45:00        LH1426                  Frankfurt Airport   
2    2025-09-01T15:10:00        AF1024  Paris - Charles de Gaulle Airport   

           airline         aircraft icao_code  
0  Royal Air Maroc   Boeing 737-800      GMMN  
1        Lufthansa   Airbus A320neo      GMMN  
2       Air France  Airbus A330-200      GMMN  

✅ ¡Datos de 3 vuelos transferidos exitosamente a MySQL en la tabla 'flight_arrival'!


In [7]:
# 5. RECOLECCIÓN, EXTRACCIÓN Y TRANSFORMACIÓN (FALLARÁ POR EL ERROR 403)
import requests
import json
import pandas as pd
import sqlalchemy
import pymysql

# Mi CLAVE DE RAPIDAPI (Misma que ya falló)
flights_key = "6398d1fe6d6fd8f80d2c914187fdcdp17f1jftj5ni15dei5b173" 
icao_casablanca = "GMMN" 
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao_casablanca}/2025-09-01T08:00/2025-09-01T20:00"

querystring = {"withLeg":"true", "direction":"Arrival", "withCancelled":"true", "withCodeshared":"true", "withCargo":"true", "withPrivate":"true", "withLocation":"false"}
headers = {'x-rapidapi-host': "aerodatabox.p.rapidapi.com", 'x-rapidapi-key': flights_key}
response = requests.request("GET", url, headers=headers, params=querystring)
flight_arrivals_json = response.json()

if response.status_code != 200:
    print(f"❌ Error al obtener datos de la API. Código de estado: {response.status_code}")
    print(flight_arrivals_json) # Mostrará el error 403

else:
    # Si milagrosamente funciona (no esperamos esto), se procede a la inserción
    def flight_extraction(flight):
        return {
            'scheduled_arrival_time': flight['arrival']['scheduledTime']['local'],
            'flight_number': flight['number'],
            'from_airport': flight['departure']['airport']['name'],
            'airline': flight['airline']['name'],
            'aircraft': flight['aircraft']['model'],
            'icao_code': flight['arrival']['airport']['icao']
        }
    
    flights_data = [flight_extraction(flight) for flight in flight_arrivals_json.get('arrivals', [])]
    df_flights = pd.DataFrame(flights_data)
    df_flights['icao_code'] = icao_casablanca 
    
    print("✅ Datos de vuelos cargados en DataFrame:")
    print(df_flights.head())

    # --- CÓDIGO DE INSERCIÓN A MYSQL (MISMO QUE YA FUNCIONÓ) ---
    schema = "gans"; host = "127.0.0.1"; user = "root"; password = "PepeCimf#666"; port = 3306; TABLE_NAME = 'flight_arrival'
    con_url = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    
    try:
        engine = sqlalchemy.create_engine(con_url)
        df_flights.to_sql(TABLE_NAME, if_exists='append', con=engine, index=False)
        print(f"\n✅ ¡Datos de {len(df_flights)} vuelos transferidos exitosamente a MySQL en la tabla '{TABLE_NAME}'!")
    except Exception as e:
        print(f"\n❌ ¡ERROR en la transferencia a MySQL! Asegúrate que tu servidor esté encendido.")
        print(f"Detalle del error: {e}")

❌ Error al obtener datos de la API. Código de estado: 403
{'message': 'You are not subscribed to this API.'}


In [8]:
# 6. DATOS SIMULADOS PARA POBLAR LA TABLA flight_arrival
import pandas as pd
import sqlalchemy
import pymysql
from numpy import nan

# Datos de Vuelos de Ejemplo para Casablanca (GMMN) - Conjunto de 10 registros
flights_data = [
    {'scheduled_arrival_time': '2025-09-01T08:00:00', 'flight_number': 'TK1297', 'from_airport': 'Istanbul Airport', 'airline': 'Turkish Airlines', 'aircraft': 'Airbus A330-300', 'icao_code': 'GMMN'},
    {'scheduled_arrival_time': '2025-09-01T08:45:00', 'flight_number': 'AT901', 'from_airport': 'Jeddah International', 'airline': 'Royal Air Maroc', 'aircraft': 'Boeing 787-8', 'icao_code': 'GMMN'},
    {'scheduled_arrival_time': '2025-09-01T09:15:00', 'flight_number': 'BA2664', 'from_airport': 'London Gatwick Airport', 'airline': 'British Airways', 'aircraft': 'Airbus A320', 'icao_code': 'GMMN'},
    {'scheduled_arrival_time': '2025-09-01T10:30:00', 'flight_number': 'AT675', 'from_airport': 'Paris - Orly Airport', 'airline': 'Royal Air Maroc', 'aircraft': 'Boeing 737-800', 'icao_code': 'GMMN'},
    {'scheduled_arrival_time': '2025-09-01T11:00:00', 'flight_number': 'IB8700', 'from_airport': 'Madrid Barajas Airport', 'airline': 'Iberia', 'aircraft': 'Airbus A321', 'icao_code': 'GMMN'},
    {'scheduled_arrival_time': '2025-09-01T12:45:00', 'flight_number': 'LH1426', 'from_airport': 'Frankfurt Airport', 'airline': 'Lufthansa', 'aircraft': 'Airbus A320neo', 'icao_code': 'GMMN'},
    {'scheduled_arrival_time': '2025-09-01T14:30:00', 'flight_number': 'W2301', 'from_airport': 'Nice Côte d''Azur Airport', 'airline': 'Wizz Air', 'aircraft': 'Airbus A321neo', 'icao_code': 'GMMN'},
    {'scheduled_arrival_time': '2025-09-01T15:10:00', 'flight_number': 'AF1024', 'from_airport': 'Paris - Charles de Gaulle Airport', 'airline': 'Air France', 'aircraft': 'Airbus A330-200', 'icao_code': 'GMMN'},
    {'scheduled_arrival_time': '2025-09-01T16:50:00', 'flight_number': 'KL201', 'from_airport': 'Amsterdam Schiphol Airport', 'airline': 'KLM', 'aircraft': 'Boeing 737-900', 'icao_code': 'GMMN'},
    {'scheduled_arrival_time': '2025-09-01T17:35:00', 'flight_number': 'FR8001', 'from_airport': 'Bergamo Airport', 'airline': 'Ryanair', 'aircraft': 'Boeing 737-800', 'icao_code': 'GMMN'}
]

# Crear el DataFrame directamente desde los datos simulados
df_flights = pd.DataFrame(flights_data)

print("✅ Datos de 10 vuelos simulados cargados en DataFrame:")
print(df_flights.head(10))


# 7. CONEXIÓN Y CARGA A MYSQL

# --- TUS CREDENCIALES DE MYSQL ---
schema = "gans"  
host = "127.0.0.1"
user = "root"
password = "PepeCimf#666" 
port = 3306
TABLE_NAME = 'flight_arrival'

# Crear la cadena de conexión con el conector PyMySQL
con_url = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

try:
    # Crear el motor de conexión
    engine = sqlalchemy.create_engine(con_url)
    
    # Enviar los datos de vuelos a la base de datos
    # NOTA: Usamos 'append' para agregar los nuevos 10 vuelos a los 3 que ya tienes
    df_flights.to_sql(TABLE_NAME, 
              if_exists='append', 
              con=engine, 
              index=False)
    
    print(f"\n✅ ¡Datos de {len(df_flights)} vuelos transferidos exitosamente a MySQL en la tabla '{TABLE_NAME}'!")
    
except Exception as e:
    print(f"\n❌ ¡ERROR en la transferencia a MySQL! Asegúrate que tu servidor esté encendido.")
    print(f"Detalle del error: {e}")

✅ Datos de 10 vuelos simulados cargados en DataFrame:
  scheduled_arrival_time flight_number                       from_airport  \
0    2025-09-01T08:00:00        TK1297                   Istanbul Airport   
1    2025-09-01T08:45:00         AT901               Jeddah International   
2    2025-09-01T09:15:00        BA2664             London Gatwick Airport   
3    2025-09-01T10:30:00         AT675               Paris - Orly Airport   
4    2025-09-01T11:00:00        IB8700             Madrid Barajas Airport   
5    2025-09-01T12:45:00        LH1426                  Frankfurt Airport   
6    2025-09-01T14:30:00         W2301            Nice Côte dAzur Airport   
7    2025-09-01T15:10:00        AF1024  Paris - Charles de Gaulle Airport   
8    2025-09-01T16:50:00         KL201         Amsterdam Schiphol Airport   
9    2025-09-01T17:35:00        FR8001                    Bergamo Airport   

            airline         aircraft icao_code  
0  Turkish Airlines  Airbus A330-300      GMMN  


In [9]:
# 8. DATOS SIMULADOS PARA POBLAR LA TABLA flight_arrival (10 Vuelos)
import pandas as pd
import sqlalchemy
import pymysql

# Datos de Vuelos de Ejemplo para Casablanca (GMMN) - 10 registros
flights_data = [
    {'flight_number': 'TK1297', 'from_airport': 'Istanbul Airport', 'airline': 'Turkish Airlines', 'aircraft': 'Airbus A330-300', 'scheduled_arrival_time_utc': '2025-09-01T07:00:00', 'scheduled_arrival_time_local': '2025-09-01T08:00:00', 'icao_code': 'GMMN'},
    {'flight_number': 'AT901', 'from_airport': 'Jeddah International', 'airline': 'Royal Air Maroc', 'aircraft': 'Boeing 787-8', 'scheduled_arrival_time_utc': '2025-09-01T07:45:00', 'scheduled_arrival_time_local': '2025-09-01T08:45:00', 'icao_code': 'GMMN'},
    {'flight_number': 'BA2664', 'from_airport': 'London Gatwick Airport', 'airline': 'British Airways', 'aircraft': 'Airbus A320', 'scheduled_arrival_time_utc': '2025-09-01T08:15:00', 'scheduled_arrival_time_local': '2025-09-01T09:15:00', 'icao_code': 'GMMN'},
    {'flight_number': 'AT675', 'from_airport': 'Paris - Orly Airport', 'airline': 'Royal Air Maroc', 'aircraft': 'Boeing 737-800', 'scheduled_arrival_time_utc': '2025-09-01T09:30:00', 'scheduled_arrival_time_local': '2025-09-01T10:30:00', 'icao_code': 'GMMN'},
    {'flight_number': 'IB8700', 'from_airport': 'Madrid Barajas Airport', 'airline': 'Iberia', 'aircraft': 'Airbus A321', 'scheduled_arrival_time_utc': '2025-09-01T10:00:00', 'scheduled_arrival_time_local': '2025-09-01T11:00:00', 'icao_code': 'GMMN'},
    {'flight_number': 'LH1426', 'from_airport': 'Frankfurt Airport', 'airline': 'Lufthansa', 'aircraft': 'Airbus A320neo', 'scheduled_arrival_time_utc': '2025-09-01T11:45:00', 'scheduled_arrival_time_local': '2025-09-01T12:45:00', 'icao_code': 'GMMN'},
    {'flight_number': 'W2301', 'from_airport': 'Nice Côte d''Azur Airport', 'airline': 'Wizz Air', 'aircraft': 'Airbus A321neo', 'scheduled_arrival_time_utc': '2025-09-01T13:30:00', 'scheduled_arrival_time_local': '2025-09-01T14:30:00', 'icao_code': 'GMMN'},
    {'flight_number': 'AF1024', 'from_airport': 'Paris - Charles de Gaulle Airport', 'airline': 'Air France', 'aircraft': 'Airbus A330-200', 'scheduled_arrival_time_utc': '2025-09-01T14:10:00', 'scheduled_arrival_time_local': '2025-09-01T15:10:00', 'icao_code': 'GMMN'},
    {'flight_number': 'KL201', 'from_airport': 'Amsterdam Schiphol Airport', 'airline': 'KLM', 'aircraft': 'Boeing 737-900', 'scheduled_arrival_time_utc': '2025-09-01T15:50:00', 'scheduled_arrival_time_local': '2025-09-01T16:50:00', 'icao_code': 'GMMN'},
    {'flight_number': 'FR8001', 'from_airport': 'Bergamo Airport', 'airline': 'Ryanair', 'aircraft': 'Boeing 737-800', 'scheduled_arrival_time_utc': '2025-09-01T16:35:00', 'scheduled_arrival_time_local': '2025-09-01T17:35:00', 'icao_code': 'GMMN'}
]

# Crear el DataFrame directamente desde los datos simulados
df_flights = pd.DataFrame(flights_data)

print("✅ Datos de 10 vuelos simulados cargados en DataFrame:")
print(df_flights.head(10))


# 9. CONEXIÓN Y CARGA A MYSQL

# --- TUS CREDENCIALES DE MYSQL ---
schema = "gans"  
host = "127.0.0.1"
user = "root"
password = "PepeCimf#666" 
port = 3306
TABLE_NAME = 'flight_arrival'

# Crear la cadena de conexión con el conector PyMySQL
con_url = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

try:
    # Crear el motor de conexión
    engine = sqlalchemy.create_engine(con_url)
    
    # Enviar los datos de vuelos a la base de datos (if_exists='append' reinsertará los 10)
    df_flights.to_sql(TABLE_NAME, 
              if_exists='append', 
              con=engine, 
              index=False)
    
    print(f"\n✅ ¡Datos de {len(df_flights)} vuelos transferidos exitosamente a MySQL en la tabla '{TABLE_NAME}'!")
    
except Exception as e:
    print(f"\n❌ ¡ERROR en la transferencia a MySQL! Asegúrate que tu servidor esté encendido.")
    print(f"Detalle del error: {e}")

✅ Datos de 10 vuelos simulados cargados en DataFrame:
  flight_number                       from_airport           airline  \
0        TK1297                   Istanbul Airport  Turkish Airlines   
1         AT901               Jeddah International   Royal Air Maroc   
2        BA2664             London Gatwick Airport   British Airways   
3         AT675               Paris - Orly Airport   Royal Air Maroc   
4        IB8700             Madrid Barajas Airport            Iberia   
5        LH1426                  Frankfurt Airport         Lufthansa   
6         W2301            Nice Côte dAzur Airport          Wizz Air   
7        AF1024  Paris - Charles de Gaulle Airport        Air France   
8         KL201         Amsterdam Schiphol Airport               KLM   
9        FR8001                    Bergamo Airport           Ryanair   

          aircraft scheduled_arrival_time_utc scheduled_arrival_time_local  \
0  Airbus A330-300        2025-09-01T07:00:00          2025-09-01T08:00:00 